# Interactive TA Dashboard (Cufflinks + ipywidgets)


In [ ]:
import cufflinks as cf
from plotly.offline import init_notebook_mode
import ipywidgets as wd
from IPython.display import display
import yfinance as yf
import pandas as pd

init_notebook_mode(connected=True)
cf.go_offline()


In [ ]:
def ta_dashboard(asset='AAPL', start='2024-01-01', end='2024-06-30', sma_period=20, ema_period=20, add_volume=True, rsi_period=0, macd_fast=0, macd_slow=0, macd_signal=0, bb_period=0, bb_std=2.0):
    df = yf.download(asset, start=start, end=end, progress=False)
    if df is None or df.empty:
        return
    qf = cf.QuantFig(df[['Open','High','Low','Close','Volume']], title=f'{asset} TA Dashboard', legend='top')
    if add_volume:
        qf.add_volume()
    if sma_period and sma_period > 0:
        qf.add_sma(periods=int(sma_period))
    if ema_period and ema_period > 0:
        qf.add_ema(periods=int(ema_period))
    if rsi_period and rsi_period > 0:
        qf.add_rsi(periods=int(rsi_period))
    if macd_fast and macd_slow and macd_signal:
        qf.add_macd(fast_period=int(macd_fast), slow_period=int(macd_slow), signal_period=int(macd_signal))
    if bb_period and bb_period > 0:
        qf.add_bollinger_bands(periods=int(bb_period), boll_std=float(bb_std))
    return qf.iplot(asFigure=True)


In [ ]:
asset_dd = wd.Text(value='AAPL', description='Asset:')
start_dp = wd.Text(value='2024-01-01', description='Start:')
end_dp = wd.Text(value='2024-06-30', description='End:')
sma_slider = wd.IntSlider(value=20, min=0, max=200, step=1, description='SMA:')
ema_slider = wd.IntSlider(value=20, min=0, max=200, step=1, description='EMA:')
rsi_slider = wd.IntSlider(value=14, min=0, max=100, step=1, description='RSI:')
macd_fast = wd.IntSlider(value=12, min=0, max=50, step=1, description='MACD fast:')
macd_slow = wd.IntSlider(value=26, min=0, max=100, step=1, description='MACD slow:')
macd_signal = wd.IntSlider(value=9, min=0, max=50, step=1, description='MACD signal:')
bb_period = wd.IntSlider(value=20, min=0, max=200, step=1, description='BB period:')
bb_std = wd.FloatSlider(value=2.0, min=0.5, max=4.0, step=0.1, description='BB std:')
vol_toggle = wd.Checkbox(value=True, description='Volume')

controls = wd.VBox([
    asset_dd, start_dp, end_dp,
    sma_slider, ema_slider,
    rsi_slider, macd_fast, macd_slow, macd_signal,
    bb_period, bb_std,
    vol_toggle
])
out = wd.Output()

def update_plot(change=None):
    with out:
        out.clear_output(wait=True)
        fig = ta_dashboard(
            asset=asset_dd.value,
            start=start_dp.value,
            end=end_dp.value,
            sma_period=sma_slider.value,
            ema_period=ema_slider.value,
            add_volume=vol_toggle.value,
            rsi_period=rsi_slider.value,
            macd_fast=macd_fast.value,
            macd_slow=macd_slow.value,
            macd_signal=macd_signal.value,
            bb_period=bb_period.value,
            bb_std=bb_std.value,
        )
        if fig is not None:
            cf.iplot(fig)

for w in [asset_dd, start_dp, end_dp, sma_slider, ema_slider, rsi_slider, macd_fast, macd_slow, macd_signal, bb_period, bb_std, vol_toggle]:
    w.observe(update_plot, names='value')

update_plot()
display(controls, out)
